In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from IPython.display import display

In [ ]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# read data
train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')

# 单独保存comment_text
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])


In [15]:
# 显示训练和测试数据
# display(train.head(10))
# display(test.head(10))
display(all_text.head(10))


0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
5    "\n\nCongratulations from me as well, use the ...
6         COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7    Your vandalism to the Matt Shirvington article...
8    Sorry if the word 'nonsense' was offensive to ...
9    alignment on this subject and which are contra...
Name: comment_text, dtype: object

In [ ]:
# 数据可视化


In [34]:
# word training
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word', # analyze by word
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

# Learn vocabulary and idf from training set
word_vectorizer.fit(all_text)

# Transform documents to document-term matrix.
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)


In [35]:
# train_word_features

In [ ]:
# character traing
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

# fit text 
char_vectorizer.fit(all_text)

# Transform documents to document-term matrix.
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)


In [ ]:
# 将两个稀疏矩阵合成
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])


In [ ]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

# 遍历7个分类标签
for class_name in class_names:
    # 获取每个分类标签列训练数据
    train_target = train[class_name]
    # 生成逻辑回归对象
    classifier = LogisticRegression(C=0.1, solver='sag')
    # 通过ROC-AUC评估模型得分
    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    # 保存得分
    scores.append(cv_score)
    
    print('CV score for class {} is {}'.format(class_name, cv_score))
    # fit数据
    classifier.fit(train_features, train_target)
    # 预测得分
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

# 保存输出文件
submission.to_csv('submission.csv', index=False)


In [ ]:
# submission